### Section 164.1: Native way (no libraries)

In [ ]:
"""
usage: sub <command>
commands:
status - show status
list - print list
"""
import sys
def check():
    print("status")
    return 0
if sys.argv[1:] == ['status']:
    sys.exit(check())
elif sys.argv[1:] == ['list']:
    print("list")
else:
    print(__doc__.strip())

In [1]:
%%cmd
python shell.py

Microsoft Windows [版本 10.0.16299.309]
(c) 2017 Microsoft Corporation。保留所有权利。

E:\MyFile\Jupyter\Python-Learn\Chapter 164 CLI subcommands with precise help output>python shell.py
usage: sub <command>
commands:
status - show status
list - print list

E:\MyFile\Jupyter\Python-Learn\Chapter 164 CLI subcommands with precise help output>

### Section 164.2: argparse (default help formatter)

In [ ]:
import argparse
import sys
def check():
    print("status")
    return 0
parser = argparse.ArgumentParser(prog="sub", add_help=False)
subparser = parser.add_subparsers(dest="cmd")
subparser.add_parser('status', help='show status')
subparser.add_parser('list', help='print list')
# hack to show help when no arguments supplied
if len(sys.argv) == 1:
    parser.print_help()
    sys.exit(0)
args = parser.parse_args()
if args.cmd == 'list':
    print('list')
elif args.cmd == 'status':
    sys.exit(check())

In [2]:
%%cmd
python shell2.py

Microsoft Windows [版本 10.0.16299.309]
(c) 2017 Microsoft Corporation。保留所有权利。

E:\MyFile\Jupyter\Python-Learn\Chapter 164 CLI subcommands with precise help output>python shell2.py
usage: sub {status,list} ...

positional arguments:
  {status,list}
    status       show status
    list         print list

E:\MyFile\Jupyter\Python-Learn\Chapter 164 CLI subcommands with precise help output>

### Section 164.3: argparse (custom help formatter)

In [ ]:
import argparse
import sys
class CustomHelpFormatter(argparse.HelpFormatter):
    def _format_action(self, action):
        if type(action) == argparse._SubParsersAction:
            # inject new class variable for subcommand formatting
            subactions = action._get_subactions()
            invocations = [self._format_action_invocation(a) for a in subactions]
            self._subcommand_max_length = max(len(i) for i in invocations)
        if type(action) == argparse._SubParsersAction._ChoicesPseudoAction:
            # format subcommand help line
            subcommand = self._format_action_invocation(action) # type: str
            width = self._subcommand_max_length
            help_text = ""
            if action.help:
                help_text = self._expand_help(action)
            return " {:{width}} - {}\n".format(subcommand, help_text, width=width)
        elif type(action) == argparse._SubParsersAction:
            # process subcommand help section
            msg = '\n'
            for subaction in action._get_subactions():
                msg += self._format_action(subaction)
            return msg
        else:
            return super(CustomHelpFormatter, self)._format_action(action)
def check():
    print("status")
    return 0
parser = argparse.ArgumentParser(usage="sub <command>", add_help=False, formatter_class=CustomHelpFormatter)
subparser = parser.add_subparsers(dest="cmd")
subparser.add_parser('status', help='show status')
subparser.add_parser('list', help='print list')
# custom help messge
parser._positionals.title = "commands"
# hack to show help when no arguments supplied
if len(sys.argv) == 1:
    parser.print_help()
    sys.exit(0)
args = parser.parse_args()
if args.cmd == 'list':
    print('list')
elif args.cmd == 'status':
    sys.exit(check())

In [3]:
%%cmd
python shell3.py

Microsoft Windows [版本 10.0.16299.309]
(c) 2017 Microsoft Corporation。保留所有权利。

E:\MyFile\Jupyter\Python-Learn\Chapter 164 CLI subcommands with precise help output>python shell3.py
usage: sub <command>

commands:

 status - show status
 list   - print list

E:\MyFile\Jupyter\Python-Learn\Chapter 164 CLI subcommands with precise help output>